# 2 tenacity中的常用功能

## 2.1 tenacity的基础使用

In [31]:
import random
from tenacity import retry

@retry
def demo_func1():

    a = random.random()
    print(a)
    
    if a >= 0.1:
        raise Exception

demo_func1()

0.554196440452871
0.37515644100597656
0.3911017511444177
0.7335292512239636
0.982713130197331
0.3973483360857295
0.12474681174071822
0.12057332993706815
0.39418215514006516
0.3191062193175437
0.11929202864244703
0.01649115465649542


## 2.2 设置最大重试次数

In [35]:
from tenacity import retry, stop_after_attempt

@retry(stop=stop_after_attempt(3))
def demo_func2():
    
    print('函数执行')
    
    raise Exception
    
demo_func2()

函数执行
函数执行
函数执行


RetryError: RetryError[<Future at 0x1a9b054b220 state=finished raised Exception>]

## 2.3 设置重试最大超时时长

In [39]:
import time
from tenacity import retry, stop_after_delay

# 设置重试最大超时时长为5秒
@retry(stop=stop_after_delay(5))
def demo_func3():
    
    time.sleep(1)
    print(f'已过去 {time.time() - start_time} 秒')
    
    raise Exception

# 记录开始时间
start_time = time.time()
demo_func3()

已过去 1.0141007900238037 秒
已过去 2.026146173477173 秒
已过去 3.033085346221924 秒
已过去 4.042022466659546 秒
已过去 5.050917148590088 秒


RetryError: RetryError[<Future at 0x1a9b0852730 state=finished raised Exception>]

## 2.4 组合重试停止条件

In [42]:
import time
import random
from tenacity import retry, stop_after_delay, stop_after_attempt

@retry(stop=(stop_after_delay(3) | stop_after_attempt(5)))
def demo_func4():
    
    time.sleep(random.random())
    print(f'已过去 {time.time() - start_time} 秒')
    
    raise Exception

# 记录开始时间
start_time = time.time()
demo_func4()

已过去 0.15338873863220215 秒
已过去 0.5147004127502441 秒
已过去 1.464496374130249 秒
已过去 1.6237852573394775 秒
已过去 1.8902502059936523 秒


RetryError: RetryError[<Future at 0x1a9ae80bfd0 state=finished raised Exception>]

## 2.5 设置相邻重试之间的时间间隔

### 2.5.1 设置固定时间间隔

In [45]:
import time
from tenacity import retry, wait_fixed, stop_after_attempt

# 设置重试等待间隔为1秒
@retry(wait=wait_fixed(1), stop=stop_after_attempt(3))
def demo_func5():
    
    print(f'已过去 {time.time() - start_time} 秒')
    
    raise Exception
    
# 记录开始时间
start_time = time.time()
demo_func5()

已过去 0.0 秒
已过去 1.0010271072387695 秒
已过去 2.011207342147827 秒


RetryError: RetryError[<Future at 0x1a9b0b3a220 state=finished raised Exception>]

### 2.5.2 设置随机时间间隔

In [48]:
import time
from tenacity import retry, wait_random, stop_after_attempt

# 设置重试等待间隔为1到3之间的随机数
@retry(wait=wait_random(min=1, max=3), stop=stop_after_attempt(5))
def demo_func6():
    
    print(f'已过去 {time.time() - start_time} 秒')
    
    raise Exception

# 记录开始时间
start_time = time.time()
demo_func6()

已过去 0.0 秒
已过去 2.488474130630493 秒
已过去 4.293655872344971 秒
已过去 7.003846645355225 秒
已过去 9.442596435546875 秒


RetryError: RetryError[<Future at 0x1a9b0b9ac40 state=finished raised Exception>]

## 2.6 自定义是否触发重试

### 2.6.1 捕捉或忽略特定的错误类型

In [66]:
from tenacity import retry, retry_if_exception_type, retry_if_not_exception_type

@retry(retry=retry_if_exception_type(FileExistsError))
def demo_func7():
    
    raise TimeoutError
    
@retry(retry=retry_if_not_exception_type(FileNotFoundError))
def demo_func8():

    raise FileNotFoundError

In [67]:
demo_func7()

TimeoutError: 

In [69]:
demo_func8()

FileNotFoundError: 

In [50]:
demo_func8()

FileNotFoundError: 

### 2.6.2 自定义函数结果条件判断函数

In [74]:
import random
from tenacity import retry, retry_if_result

@retry(retry=retry_if_result(lambda x: x >= 0.1))
def demo_func9():
    a = random.random()
    print(a)
    return a

# 记录开始时间
demo_func9()

0.9910761438314104
0.8953414949545078
0.2013066988816824
0.8352827649284115
0.2644339159107133
0.3246973862900614
0.022107955836089266


0.022107955836089266

## 2.7 对函数的错误重试情况进行统计

In [76]:
demo_func9.retry.statistics

{'start_time': 9668.718,
 'attempt_number': 7,
 'idle_for': 0,
 'delay_since_first_attempt': 0.0}